<a href="https://colab.research.google.com/github/hmatny/question_answering/blob/master/Anserini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment Setup

#### Download and install maven

In [1]:
!wget http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
!tar -xvzf apache-maven-3.6.1-bin.tar.gz -C /opt

--2019-04-18 04:44:19--  http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
Resolving mirror.reverse.net (mirror.reverse.net)... 208.100.14.200
Connecting to mirror.reverse.net (mirror.reverse.net)|208.100.14.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9136463 (8.7M) [application/x-gzip]
Saving to: ‘apache-maven-3.6.1-bin.tar.gz.1’

apache-maven-3.6.1- 100%[===================>]   8.71M  4.89MB/s    in 1.8s    

2019-04-18 04:44:21 (4.89 MB/s) - ‘apache-maven-3.6.1-bin.tar.gz.1’ saved [9136463/9136463]

apache-maven-3.6.1/README.txt
apache-maven-3.6.1/LICENSE
apache-maven-3.6.1/NOTICE
apache-maven-3.6.1/lib/
apache-maven-3.6.1/lib/slf4j-api.license
apache-maven-3.6.1/lib/checker-compat-qual.license
apache-maven-3.6.1/lib/jsr250-api.license
apache-maven-3.6.1/lib/jcl-over-slf4j.license
apache-maven-3.6.1/lib/org.eclipse.sisu.plexus.license
apache-maven-3.6.1/lib/animal-sniffer-annotations.license
apache-maven-

#### Set environment variables

In [2]:
import os
os.environ['PATH'] += ':/usr/lib/jvm/java-8-openjdk-amd64/bin:/opt/apache-maven-3.6.1/bin'
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['M2_HOME'] = "/opt/apache-maven-3.6.0"

# sanity check
!mvn -version

Apache Maven 3.6.1 (d66c9c0b3152b2e69ee9bac180bb8fcc8e6af555; 2019-04-04T19:00:29Z)
Maven home: /opt/apache-maven-3.6.1
Java version: 1.8.0_191, vendor: Oracle Corporation, runtime: /usr/lib/jvm/java-8-openjdk-amd64/jre
Default locale: en_US, platform encoding: UTF-8
OS name: "linux", version: "4.14.79+", arch: "amd64", family: "unix"


#### Clone Anserini git repo

In [3]:
!git clone https://github.com/castorini/Anserini.git
!cd Anserini && mvn clean package appassembler:assemble

fatal: destination path 'Anserini' already exists and is not an empty directory.
[INFO] Scanning for projects...
[INFO] 
[INFO] ------------------------< io.anserini:anserini >------------------------
[INFO] Building Anserini 0.4.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-clean-plugin:2.5:clean (default-clean) @ anserini ---
[INFO] Deleting /content/Anserini/target
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ anserini ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 104 resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.7.0:compile (default-compile) @ anserini ---
[INFO] Changes detected - recompiling the module!
[INFO] Compiling 128 source files to /content/Anserini/target/classes
[WARNING] /content/Anserini/src/main/java/io/anserini/analysis/EnglishStemmingAnalyzer.java:[25,43] org.apache.lucene.analysis.standard.StandardFilter in org.apache.lucene.a

#### Get CAR17 corpus and index data

In [0]:
# !wget http://trec-car.cs.unh.edu/datareleases/v2.0/paragraphCorpus.v2.0.tar.xz
# !tar -xvf paragraphCorpus.v2.0.tar.xz -C /content/Anserini/

In [16]:
# !cd /content/Anserini/ && sh target/appassembler/bin/IndexCollection -collection CarCollection \
# -generator LuceneDocumentGenerator -threads 500000 -input /content/Anserini/paragraphCorpus -index \
# lucene-index.car17a.pos+docvectors+rawdocs -storePositions -storeDocvectors \
# -storeRawDocs

2019-04-18 04:53:13,704 INFO  [main] index.IndexCollection (IndexCollection.java:429) - DocumentCollection path: /content/Anserini/paragraphCorpus
2019-04-18 04:53:13,705 INFO  [main] index.IndexCollection (IndexCollection.java:430) - Index path: lucene-index.car17a.pos+docvectors+rawdocs
2019-04-18 04:53:13,705 INFO  [main] index.IndexCollection (IndexCollection.java:431) - CollectionClass: CarCollection
2019-04-18 04:53:13,705 INFO  [main] index.IndexCollection (IndexCollection.java:432) - Generator: LuceneDocumentGenerator
2019-04-18 04:53:13,705 INFO  [main] index.IndexCollection (IndexCollection.java:433) - Threads: 500000
2019-04-18 04:53:13,706 INFO  [main] index.IndexCollection (IndexCollection.java:434) - Stemmer: porter
2019-04-18 04:53:13,706 INFO  [main] index.IndexCollection (IndexCollection.java:435) - Keep stopwords? false
2019-04-18 04:53:13,706 INFO  [main] index.IndexCollection (IndexCollection.java:436) - Store positions? true
2019-04-18 04:53:13,706 INFO  [main] ind

In [0]:
# tar index
# !cd /content/Anserini/ && tar czf lucene-index.car17-large.tar.gz lucene-index.car17a.pos+docvectors+rawdocs

In [22]:
# save index to gs bucket
# !gsutil cp /content/Anserini/lucene-index.car17-large.tar.gz gs://cs378_bert/lucene-index-large.tar.gz

Copying file:///content/Anserini/lucene-index.car17-large.tar.gz [Content-Type=application/x-tar]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/14.1 GiB.                                     


#### Download the Lucene index

In [0]:
!gsutil cp gs://cs378_bert/lucene-index-large.tar.gz /tmp/lucene-index.tar.gz
!cd /tmp && tar -xvzf lucene-index-large.tar.gz -C /content/Anserini/

#### Install dependencies

In [23]:
!pip3 install pyjnius

### Using Anserini 

In [0]:
# Only run this once
import jnius_config
jnius_config.set_classpath('Anserini/target/anserini-0.4.1-SNAPSHOT-fatjar.jar')

In [26]:
from jnius import autoclass
JString = autoclass('java.lang.String')
JSearcher = autoclass('io.anserini.search.SimpleSearcher')

searcher = JSearcher(JString('Anserini/lucene-index.car17a.pos+docvectors+rawdocs'))
hits = searcher.search(JString('Who is Albert Einstein?'))

for ind, hit in enumerate(hits):
        print('hit #' + str(ind))
        print('docid: ' + str(hit.docid))
        print('score: ' + str(hit.score))
        print('content: ')
        print(hit.content)
        print()

hit #0
docid: 1a23029cda5ae298e03b91ec28949e462d2b432b
score: 11.577500343322754
content: 
Bernard Einstein was the son of Hans Albert Einstein and Frieda Einstein (née Knecht), who had married in 1927 in Switzerland. He was born on 10 July 1930 in Dortmund, Germany where Hans Albert was involved in a bridge building project. Hans Albert was the only one of Albert Einstein's three children to marry and have children. Hans Albert's sister, Lieserl Einstein, is assumed to have died in infancy, although it has been suggested that she was given up for adoption. Hans Albert's younger brother Eduard Einstein developed severe schizophrenia in early adulthood and died at age 55, and had no children. Einstein's brother, Klaus, was born in 1932 and died at age 6 of diphtheria in Clemson, South Carolina, United States. Frieda had David who died in early infancy. Eventually Hans Albert and Frieda adopted a girl, Evelyn, shortly after she was born in 1941. One year after Frieda died in 1958, Hans A